# Question 3

## 1. Identify all inconsistent data formats and correct them

The first and most visible data quality issue in the dataset appears in the **Timestamp column**, where the same hourly time series is represented using several different string formats. Instead of following a single date-time structure, the dataset mixes multiple conventions for separators, ordering, time notation, and precision. For example, some timestamps follow a day-month-year pattern with AM/PM notation, such as “01-01-2025 12:00AM”, while others use a year-month-day structure with slashes and full seconds, such as “2025/01/01 01:00:00”. Additional rows use a 24-hour clock without seconds (“2025-01-01 02:00”) or the day-month-year format with AM/PM (“01-01-2025 03:00AM”).

The dataset also includes European-style entries such as “01/01/2025 09:00” and even ambiguous forms like “02/01/2025 06:00”, where the meaning of “02/01” depends on whether the parser assumes a day-first or month-first convention. Together, these inconsistencies mean that the timestamps cannot be reliably sorted, compared, or resampled until they are standardized into a single coherent datetime format.

### Performing EDA before starting data preprocessing

In [8]:
# Basic structure fo the dataset
file_path = "../datasets/Dataset_for_Q3.csv"   # <== Your uploaded file
df_q3 = pd.read_csv(file_path)

print("\n################## Shape of dataset is:##################  \n", df_q3.shape)
print("\n################## Columns of dataset are:##################  \n", df_q3.columns.tolist())
print("\n################## Top 5 rows of dataset are:##################  \n", df_q3.head(5))
print("\n################## Last 5 rows of dataset are:##################  \n", df_q3.tail(5))

# Check for missing values
print("\n Number of missing values: \n", df_q3.isna().sum())


# Summary statistics
print("\n##################Statistical Summary of the dataset: ##################\n", df_q3.describe(include="all"))
print("\n################## Dataframe Summary: ##################\n")
df.info()


################## Shape of dataset is:##################  
 (50, 6)

################## Columns of dataset are:##################  
 ['Timestamp', 'Temperature', 'TempUnit', 'Humidity', 'Airflow', 'StatusText']

################## Top 5 rows of dataset are:##################  
              Timestamp  Temperature TempUnit  Humidity  Airflow StatusText
0   01-01-2025 12:00AM        26.11        C     67.18   127.16         OK
1  2025/01/01 01:00:00        85.63        F     51.11   131.22     Sta#us
2     2025-01-01 02:00        35.68        C     50.57   141.66         OK
3   01-01-2025 03:00AM        84.53        F     58.78   141.08         OK
4   01-01-2025 04:00AM        28.37        C     74.34    92.45     Active

################## Last 5 rows of dataset are:##################  
               Timestamp  Temperature TempUnit  Humidity  Airflow StatusText
45  2025/01/03 07:00:00        27.77        C     43.88   113.31     Active
46  2025/01/03 08:00:00        87.65        F   

In [18]:
# ---------------------------------------------------------
# 1. Load the dataset
# ---------------------------------------------------------
file_path = "../datasets/Dataset_for_Q3.csv"
df_q3 = pd.read_csv(file_path)

print("Original Data Preview:")
print(df_q3.head())

# ---------------------------------------------------------
# 2. Multi-format Timestamp Parser & Replacement
# ---------------------------------------------------------
timestamp_formats = [
    "%d-%m-%Y %I:%M%p",     # 01-01-2025 12:00AM
    "%Y/%m/%d %H:%M:%S",    # 2025/01/01 01:00:00
    "%Y-%m-%d %H:%M",       # 2025-01-01 02:00
    "%d/%m/%Y %H:%M",       # 01/01/2025 09:00
    "%d-%m-%Y %H:%M",       # 01-01-2025 04:00
]

def parse_timestamp(ts):
    for fmt in timestamp_formats:
        try:
            return datetime.strptime(ts, fmt)
        except:
            continue
    return pd.NaT

# 1. Parse dates into a temporary object column for sorting
df_q3["Temp_Date_Obj"] = df_q3["Timestamp"].apply(parse_timestamp)

# 2. Sort the data chronologically
df_q3 = df_q3.sort_values("Temp_Date_Obj").reset_index(drop=True)

# 3. OVERWRITE the original 'Timestamp' column with the clean string format
df_q3["Timestamp"] = df_q3["Temp_Date_Obj"].dt.strftime("%Y-%m-%d %H:%M:%S")

# 4. Remove the temporary object column
df_q3.drop(columns=["Temp_Date_Obj"], inplace=True)

# ---------------------------------------------------------
# 3. Clean StatusText & Replacement
# ---------------------------------------------------------
# 1. Create a normalized series (Internal use only)
status_normalized = (
    df_q3["StatusText"]
        .astype(str)
        .str.normalize("NFKD")
        .str.encode("ascii", errors="ignore")
        .str.decode("utf-8")
        .str.replace(r"[^A-Za-z]", "", regex=True)
        .str.strip()
        .str.title()
)

# 2. Define the mapping
status_map = {
    "Ok": "OK",
    "Status": "OK",
    "Staus": "OK",
    "Ruing": "Running",
    "Active": "Active",
    "Running": "Running"
}

# 3. OVERWRITE the original 'StatusText' column with the mapped values
df_q3["StatusText"] = status_normalized.map(status_map)

# ---------------------------------------------------------
# 4. Final Preview
# ---------------------------------------------------------
print("\nFinal Cleaned Dataset (Old columns replaced):")
print(df_q3.head(10))

# ---------------------------------------------------------
# 5. Save cleaned dataset
# ---------------------------------------------------------
output_path = "../datasets/output/Dataset_for_Q3_cleaned.csv"
df_q3.to_csv(output_path, index=False)

print(f"\nCleaned dataset saved to: {output_path}")

Original Data Preview:
             Timestamp  Temperature TempUnit  Humidity  Airflow StatusText
0   01-01-2025 12:00AM        26.11        C     67.18   127.16         OK
1  2025/01/01 01:00:00        85.63        F     51.11   131.22     Sta#us
2     2025-01-01 02:00        35.68        C     50.57   141.66         OK
3   01-01-2025 03:00AM        84.53        F     58.78   141.08         OK
4   01-01-2025 04:00AM        28.37        C     74.34    92.45     Active

Final Cleaned Dataset (Old columns replaced):
             Timestamp  Temperature TempUnit  Humidity  Airflow StatusText
0  2025-01-01 00:00:00        26.11        C     67.18   127.16         OK
1  2025-01-01 01:00:00        85.63        F     51.11   131.22         OK
2  2025-01-01 02:00:00        35.68        C     50.57   141.66         OK
3  2025-01-01 03:00:00        84.53        F     58.78   141.08         OK
4  2025-01-01 04:00:00        28.37        C     74.34    92.45     Active
5  2025-01-01 05:00:00        

## 2. Inspecting the Time Sequence and Identifying Missing Hourly Timestamps

After standardizing all timestamps in the Timestamp_clean column, the dataset was evaluated for temporal completeness. Since the sensor should report hourly, every hour between the earliest and latest timestamps should appear at least once. To check this, I generated a full hourly timeline and compared it with the actual recorded timestamps.

This comparison revealed 1,376 missing hourly readings, indicating significant gaps in reporting. Examples of missing timestamps early in the series include:
- 2025-01-01 10:00:00
- 2025-01-01 11:00:00
- 2025-01-01 12:00:00
- 2025-01-01 13:00:00
- 2025-01-01 14:00:00

These gaps occur within what should be a continuous hourly sequence and must be addressed or flagged before performing downstream time-series analysis or anomaly detection.

In [10]:
# import cleaned dataset

clean_data = "../datasets/output/Dataset_for_Q3_cleaned.csv"
df_q3_cleaned = pd.read_csv(clean_data)

# Ensure Timestamp_clean is sorted
df_q3_cleaned = df_q3_cleaned.sort_values("Timestamp_clean")

# Build the full expected hourly timeline
full_range = pd.date_range(
    start=df_q3_cleaned["Timestamp_clean"].min(),
    end=df_q3_cleaned["Timestamp_clean"].max(),
    freq="h"
)

# Identify missing timestamps
missing_timestamps = full_range.difference(df_q3_cleaned["Timestamp_clean"])

print("Missing hourly timestamps:")
print(missing_timestamps[:10])   # show first 10

Missing hourly timestamps:
DatetimeIndex(['2025-01-01 00:00:00', '2025-01-01 01:00:00',
               '2025-01-01 02:00:00', '2025-01-01 03:00:00',
               '2025-01-01 04:00:00', '2025-01-01 05:00:00',
               '2025-01-01 06:00:00', '2025-01-01 07:00:00',
               '2025-01-01 08:00:00', '2025-01-01 09:00:00'],
              dtype='datetime64[ns]', freq='h')


### 3. Identifying Mixed Temperature Units and Standardizing Them
The dataset contains temperatures recorded in both Celsius and Fahrenheit, which makes the series inconsistent. To standardize the data, all Fahrenheit values were converted to Celsius using
$$
T_C = (T_F - 32) \times \frac{5}{9}
$$
This ensures all readings are on the same scale for accurate analysis.

In [20]:
# Convert Fahrenheit -> Celsius
df_q3_cleaned["Temperature_C"] = df_q3_cleaned.apply(
    lambda row: (row["Temperature"] - 32) * 5/9 if row["TempUnit"] == "F"
                else row["Temperature"],
    axis=1
)

# Optional: round for readability
df_q3_cleaned["Temperature_C"] = df_q3_cleaned["Temperature_C"].round(2)

print(df_q3_cleaned[["Temperature", "TempUnit", "Temperature_C"]].head(10))

   Temperature TempUnit  Temperature_C
0        26.11        C          26.11
1        85.63        F          29.79
2        35.68        C          35.68
3        84.53        F          29.18
4        28.37        C          28.37
5        26.45        C          26.45
6        26.20        C          26.20
7        30.84        C          30.84
8        83.97        F          28.87
9        26.09        C          26.09


### 4. Why Mixed Temperature Units Create False Anomaly Signals

Mixing Celsius and Fahrenheit values in the same time series creates sudden jumps that are not real temperature changes. For example, a normal reading like 85°F (~29°C) appears as an extreme spike if treated as 85°C. These artificial jumps distort statistical measures, trigger false outliers, and make anomaly-detection models misinterpret normal behavior as faults or overheating. Converting all readings to a single unit prevents these misleading signals.

### 5. Fahrenheit value that could be misinterpreted as an extreme outlier

A clear example is the reading:

132.77 °F at 2025-01-01 23:00

If interpreted as Celsius instead of Fahrenheit, it would appear to be an impossible 132.77°C—a massive outlier compared to the normal 25–30°C range.
After conversion, however:

<p align="center"><b>132.77&nbsp;°F &approx; 55.98&nbsp;°C</b></p>


Still high, but not an extreme error—showing why unit mixing can falsely trigger anomaly alerts.

In [39]:
# 1. Filter only Fahrenheit readings
df_f = df_q3[df_q3["TempUnit"] == "F"].copy()

# 2. Identify the single Fahrenheit value that is the highest
#    (this is the one most likely to be mistaken as a Celsius outlier)
outlier_row = df_f.loc[df_f["Temperature"].idxmax()]

# 3. Extract the temperature and timestamp
temp_f = outlier_row["Temperature"]
timestamp = outlier_row["Timestamp"]

# 4. Convert to Celsius
temp_c = (temp_f - 32) * 5/9

print("Fahrenheit value most likely to be misinterpreted as a Celsius outlier:")
print(outlier_row)

print(f"\nCorrect conversion:")
print(f"{temp_f:.2f} °F → {temp_c:.2f} °C")
print(f"Timestamp: {timestamp}")


Fahrenheit value most likely to be misinterpreted as a Celsius outlier:
Timestamp      2025-01-01 23:00:00
Temperature                 132.77
TempUnit                         F
Humidity                      30.3
Airflow                     133.03
StatusText                 Running
Name: 13, dtype: object

Correct conversion:
132.77 °F → 55.98 °C
Timestamp: 2025-01-01 23:00:00


### 6. By inspecting the temperature values, identify three single-timestamp spikes that do not align with normal sensor behavior

By reviewing the temperature values (after converting all Fahrenheit readings to Celsius), three clear one-off spikes stand out. These values are far above the surrounding observations and do not follow normal sensor behavior:

- 2025-01-01 23:00 — 132.77 °F ≈ 55.98 °C
- 2025-01-02 09:00 — 129.65 °F ≈ 54.25 °C
- 2025-01-02 23:00 — 65.30 °C

Each of these occurs only once and is surrounded by much lower values (~24–32 °C), indicating they are isolated spikes likely caused by sensor error, transient noise, or corrupted readings rather than genuine environmental changes.

In [13]:
# Assume df_q3_cleaned already loaded and has Timestamp_clean, Temperature, TempUnit

# Convert all temperatures to Celsius
def to_c(row):
    t = row["Temperature"]
    if row["TempUnit"] == "F":
        return (t - 32) * 5 / 9
    return t

df_q3_cleaned["Temp_C"] = df_q3_cleaned.apply(to_c, axis=1)

# Compute z-scores for temperature in °C
mean = df_q3_cleaned["Temp_C"].mean()
std = df_q3_cleaned["Temp_C"].std()
df_q3_cleaned["z_temp"] = (df_q3_cleaned["Temp_C"] - mean) / std

# Pick large, isolated spikes (|z| > 3 as a simple criterion)
spikes = df_q3_cleaned.loc[df_q3_cleaned["z_temp"].abs() > 3,
                   ["Timestamp_clean", "Temperature", "TempUnit", "Temp_C", "z_temp"]]

print(spikes)

        Timestamp_clean  Temperature TempUnit     Temp_C    z_temp
13  2025-01-01 23:00:00       132.77        F  55.983333  3.290677
22  2025-01-02 09:00:00       129.65        F  54.250000  3.069553
36  2025-01-02 23:00:00        65.30        C  65.300000  4.479217


## 7. Methods for Handling Single-Timestamp Spikes
Method 1 – Correct or Replace (Imputation / Smoothing)

### Method 1 — Remove/Correct the Spike Using Interpolation

Description:
When an anomaly is clearly a sensor glitch (e.g., a sudden jump from 27°C to 56°C and back), the spike can be replaced using statistical or time-based interpolation (linear, nearest, rolling median).

#### When appropriate:
- When the physical system changes gradually (HVAC, environmental monitoring).
- When the spike is mathematically impossible or violates operational limits.
- When preparing data for machine learning, forecasting, or smoothing.

#### Why:
It prevents unrealistic spikes from distorting trends, averages, and anomaly models.

### Method 2 — Keep the Value but Flag It as an Anomaly

Description:
Rather than removing the spike, the value is kept, but you add an AnomalyFlag = 1.

#### When appropriate:
- When the spike might represent a real event, such as overheating, airflow blockage, or equipment failure.
- When maintaining data integrity is important (industrial logging, safety systems).
- When analysts need to trace the real behavior of the system.

#### Why:
This preserves potentially meaningful events while still highlighting them for investigation.


## 8. Decision for Each Temperature Spike
#### Spike 1 — 132.77 °F (≈ 55.98 °C) at 2025-01-01 23:00

##### Decision: Correct (or interpolate) for analysis; keep + flag for monitoring.
##### Justification:
This is an isolated jump far outside the normal 25–30 °C pattern, indicating a probable sensor glitch. For machine-learning or trend analysis, the value should be corrected to avoid biasing the model. However, it should still be retained and flagged in operational logs in case the spike reflects a brief overheating event.

#### Spike 2 — 129.65 °F (≈ 54.25 °C) at 2025-01-02 09:00

##### Decision: Correct (interpolate).
##### Justification:
This spike occurs only at a single timestamp and returns immediately to normal. The abrupt jump is inconsistent with physical HVAC behavior, which cannot heat or cool that quickly. Treating it as a faulty reading prevents false anomaly alerts or exaggerated variability in the temperature series.

#### Spike 3 — 65.30 °C at 2025-01-02 23:00

##### Decision: Remove or correct, depending on system limits.
##### Justification:
This spike is extreme even in Celsius and does not match the trend before or after. If the monitored equipment cannot reach 65 °C under normal operation, the value should be removed as impossible. If such temperatures are physically possible (e.g., a heater or motor housing), it should be kept but flagged for further investigation.

## 9. The StatusText column contains corrupted labels
The StatusText column contained several corrupted labels, including inconsistent spelling, symbols, and accented characters (e.g., Sta#us, Ru!!ing, Açtive). To standardize these, I first normalized the raw text by removing accents, stripping non-alphabetic characters, and applying consistent capitalization. This produced intermediate forms such as Staus, Ruing, and Active. I then created a mapping table that explicitly grouped these variants into clean categories: both OK and Sta#us/Staus were mapped to “OK”, Active/Açtive to “Active”, and Running/Ru!!ing (Ruing) to “Running”. The result is a StatusClean field that provides consistent, analyzable status labels across the entire dataset.

#### This is also handled in 3. Normalization / Scaling — Best Method: Robust Scaling

In [14]:
# Normalize corrupted labels
df_q3_cleaned["StatusText_norm"] = (
    df_q3_cleaned["StatusText"]
        .astype(str)
        .str.normalize("NFKD")
        .str.encode("ascii", errors="ignore")
        .str.decode("utf-8")
        .str.replace(r"[^A-Za-z]", "", regex=True)
        .str.strip()
        .str.title()
)

# Final mapping table
# Final mapping table
status_map = {
    "Ok": "OK",             # Maps 'OK' -> 'Ok' back to 'OK'
    "Status": "OK",         # Maps 'Status' -> 'OK'
    "Staus": "OK",          # Handles 'Sta#us' -> 'Staus'
    "Ruing": "Running",     # <--- NEW: Handles 'Ru!!ing' -> 'Ruing'
    "Active": "Active",     # Handles 'Açtive' -> 'Active'
    "Running": "Running"
}
# Apply mapping
df_q3_cleaned["StatusClean"] = df_q3_cleaned["StatusText_norm"].map(status_map)

print(df_q3_cleaned[["StatusText", "StatusText_norm", "StatusClean"]])

   StatusText StatusText_norm StatusClean
0          OK              Ok          OK
1      Sta#us           Staus          OK
2          OK              Ok          OK
3          OK              Ok          OK
4      Active          Active      Active
5      Sta#us           Staus          OK
6          OK              Ok          OK
7      Sta#us           Staus          OK
8     Running         Running     Running
9      Active          Active      Active
10     Sta#us           Staus          OK
11         OK              Ok          OK
12         OK              Ok          OK
13    Ru!!ing           Ruing     Running
14     Active          Active      Active
15     Active          Active      Active
16    Ru!!ing           Ruing     Running
17     Sta#us           Staus          OK
18     Active          Active      Active
19         OK              Ok          OK
20         OK              Ok          OK
21    Running         Running     Running
22     Active          Active     

## 10. Briefly explain one potential risk of applying incorrect fuzzy text matching in an industrial monitoring or safety system.

### Risk: Semantic Misinterpretation Leading to False Negatives

Explanation: Fuzzy matching algorithms correct text based on character similarity (Levenshtein distance) without understanding context. In industrial safety, terms with opposite meanings often look similar (e.g., "Severed" vs. "Secured").

Consequence: If a sensor reports a corrupted critical error like "Se!ured" (intended to be "Severed"), an incorrect fuzzy match might map it to the benign status "Secured" or "OK." This False Negative would suppress the alarm, leading operators to believe the system is safe when a critical failure has actually occurred, potentially resulting in catastrophic equipment damage or injury.

# 11. Identify at least two duplicate airflow readings that appear across different timestamps
Identification of Duplicates: Analysis of the Airflow column reveals a specific value, 130.3, which repeats exactly across three distinct timestamps: 2025-01-01 06:00, 01-01-2025 08:00AM, and 2025-01-02 17:00

In [40]:
# 2. Find duplicates in the specific column 'Airflow'
# keep=False ensures ALL instances of the duplicate are shown, not just the extra ones.

df_airflow_duplicate = pd.read_csv("../datasets/output/Dataset_for_Q3_cleaned.csv")
duplicates = df_airflow_duplicate[df_airflow_duplicate.duplicated(subset=['Airflow'], keep=False)]

# 3. Sort by Airflow to see them grouped together
result = duplicates[['Timestamp', 'Airflow']].sort_values(by='Airflow')

print(result.head(2))

             Timestamp  Airflow
6  2025-01-01 06:00:00    130.3
8  2025-01-01 08:00:00    130.3


## 12. Decide whether each repeated value is likely a true repeated measurement or an accidental duplication, and justify your reasoning based on airflow behavior in systems.
The repeated airflow value of 130.3 is likely an accidental duplication or data artifact, not a true repeated measurement.

### Justification based on Airflow Behavior:
- **Continuous Fluctuation:** Airflow in industrial systems is a dynamic physical quantity. It is subject to constant, minute variations caused by fluid turbulence, slight changes in fan speed, and environmental resistance. In a functioning system, readings typically fluctuate by small decimals (e.g., 130.28, 130.32) rather than holding a static value.
- **Statistical Improbability:** The sensor reports data with high precision (two decimal places). The probability of a sensor independently measuring the exact same floating-point value (130.30) at three different times—separated by hours and days—is statistically negligible.
- **Sensor "Latching":** This pattern is characteristic of a sensor or data logger "freezing," where the system fails to capture a new reading and simply repeats the last known valid value (a "zero-order hold" error) or defaults to a pre-set error code value.


## Data Quality Evaluation
The data-quality issues in this IoT time series can easily distort analysis if not corrected. Inconsistent timestamp formats and long gaps in hourly readings break temporal continuity and mislead trend or anomaly detection models.

Mixed Celsius and Fahrenheit values are especially harmful because unconverted readings appear as extreme outliers, triggering false alerts. Corrupted status labels and duplicate airflow values can further bias categorical summaries or operational monitoring. Overall, mixed units and irregular timestamps are the most damaging because they alter both the scale and sequence of the signal.

Improving future data collection requires enforcing strict schemas, real-time timestamp validation, automated unit checks, and rules that flag impossible values and missing intervals at ingestion.